In [1]:
# coding: utf-8
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing, model_selection

In [ ]:
def ginic(actual, pred):
    actual = np.asarray(actual) #In case, someone passes Series or list
    n = 50
    a_s = actual[np.argsort(pred)]
    a_c = a_s.cumsum()
    giniSum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
    return giniSum / n

# ## Load Data
train = pd.read_csv("data/train.csv")
#test = pd.read_csv("data/test.csv")

#d_median = train.median(axis=0)
#d_mean = train.mean(axis=0)

def transform_df(df):
    df = pd.DataFrame(df)
    dcol = [c for c in df.columns if c not in ['id','target']]
    df['ps_car_13_x_ps_reg_03'] = df['ps_car_13'] * df['ps_reg_03']
    df['negative_one_vals'] = np.sum((df[dcol]==-1).values, axis=1)
    for c in dcol:
        if '_bin' not in c:
            df[c+str('_median_range')] = (df[c].values > d_median[c]).astype(int)
            df[c+str('_mean_range')] = (df[c].values > d_mean[c]).astype(int)
    return df

def prepare_data(dataset,Useless_fatures, name_target = 'target', replace_Nan = 0):
    dataset['ps_car_13_x_ps_reg_03'] = dataset['ps_car_13'] * dataset['ps_reg_03']
    if replace_Nan != False:
        dataset.replace(-1, replace_Nan )

    if name_target in dataset:
        X = dataset.drop([name_target],1)
        X = X.drop(Useless_fatures,1)
        
        label = pd.DataFrame()
        label['real'] = dataset[name_target]
        label['inv']  = 1-dataset[name_target]
        y = label.as_matrix()
        print(len(X), " ",len(label))
        return X , y
    else:
        return dataset.drop(Useless_fatures,1)
    


In [ ]:
def make_sub(test,model,sess,best_validation_accuracy = ''):
    print('creating submission results...')
    saver.restore(session,'models/my_model')
    #X_sub = transform_df(test)
    X_sub = prepare_data(test,['id'])
    sub1 = sess.run(model,feed_dict={x:X_sub[:int(892816/2)],keep_prob:1.0})
    sub2 = sess.run(model,feed_dict={x:X_sub[int(892816/2):],keep_prob:1.0})
    sub_tot = np.append(sub1,sub2,axis=0)
    print('Output Created!')
    max_index= sub_tot.argmax(1)
    max_res = np.amax(sub_tot,1)
    sub_tot = [n if max_index[counter] == 0 else 1-n for counter,n in zip(range(len(max_res)),max_res)]
    test_id = test.id.values    
    sub = pd.DataFrame()
    sub['id'] = test_id
    sub['target'] = sub_tot
    sub.to_csv('submission_{}.csv'.format(int(best_validation_accuracy*100)) ,index = False)
    print('Done!')

In [31]:
from sklearn.utils import shuffle
target_0 = train['target']==0
target_1 = train['target']==1
train_down_samp = train[target_1].append(train[target_0][0:int(sum(target_1)*3)])
train_down_samp = train_down_samp.append(train[target_1])
train_down_samp = train_down_samp.append(train[target_1])
#sum(new_train['target']==1)/len(new_train)

Useless_fatures = ['ps_ind_10_bin','ps_ind_11_bin','ps_ind_13_bin','ps_calc_20_bin','ps_ind_12_bin','ps_calc_15_bin','id']
#Useless_fatures = train.columns[train.columns.str.startswith('ps_calc_')] 
#new_df_train = transform_df(train_down_samp)
X_50 , y_50 = prepare_data(train_down_samp,['id'])
X_real_test, y_real_test = prepare_data(shuffle(train_down_samp)[0:3000],['id'])

X_train, X_test, y_train, y_test = model_selection.train_test_split(X_50 , y_50, test_size=0.02)

print("Size of:")
print("- Training-set:\t\t{}".format(len(X_train)))
print("- Test-set:\t\t{}".format(len(X_test)))


130164   130164
3000   3000
Size of:
- Training-set:		127560
- Test-set:		2604


In [32]:
#input size
input_size = X_train.shape[1]
num_classes = 2 #y_test.shape[1]
tf.reset_default_graph()


# ### Placeholder variables
x = tf.placeholder(tf.float32, [None, input_size])
y_true = tf.placeholder(tf.float32, [None,num_classes]) ##[None, num_classes]
y_true_cls = tf.argmax(y_true, axis=1)

# ### Helper-functions for creating new variables
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.01), name = 'weight')

def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]),name='bias')


# ### Helper-function for creating a new Fully-Connected Layer
def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs,    # Num. outputs.
                 use_relu=True,  # Use Rectified Linear Unit (ReLU)?
                 use_sig=False):
    # Create new weights and biases.
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)

    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    layer = tf.matmul(input, weights) + biases

    # Use ReLU?
    if use_relu:
        layer = tf.nn.relu(layer)
    elif use_sig:
        layer = tf.nn.sigmoid(layer)
    return layer


# ###  Layers
#hyperparameters
n_nodes_hl1 = 400
n_nodes_hl2 = 1500
n_nodes_hl3 = 500
beta = 0.0001  # l2 regularization


keep_prob = tf.placeholder(tf.float32)

layer_h1 = new_fc_layer(input=x,
                         num_inputs=input_size,
                         num_outputs=n_nodes_hl1,
                         use_relu=True)
drop_h1 = tf.nn.dropout(layer_h1, keep_prob)

layer_h2 = new_fc_layer(input=drop_h1,
                         num_inputs=n_nodes_hl1,
                         num_outputs=n_nodes_hl2,
                         use_relu=True,
                         use_sig = False)

drop_h2 = tf.nn.dropout(layer_h2, keep_prob)

layer_h3 = new_fc_layer(input=drop_h2,
                         num_inputs=n_nodes_hl2,
                         num_outputs=n_nodes_hl3,
                         use_relu=True,
                         use_sig = False)
drop_h3 = tf.nn.dropout(layer_h3, keep_prob)


y_pred = new_fc_layer(input=drop_h3,
                         num_inputs=n_nodes_hl3,
                         num_outputs=num_classes,
                         use_relu=False,
                         use_sig = False)
y_pred_soft_max = tf.nn.softmax(y_pred)
y_pred_cls = tf.argmax(y_pred, axis=1)



# ### Cost-function and Optimizer
learning_rate = tf.placeholder(tf.float32, shape=[])
#global_step = tf.Variable(0, trainable=False)
#starter_learning_rate = 0.1
#learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           #100000, 0.96, staircase=True)


##########################
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=y_pred,
                                                        labels=y_true)

#mse = tf.losses.mean_squared_error(y_true,y_pred_soft_max)


#cross_weighted =  tf.multiply(cross_entropy,y_true[:,0]*0.00005)
var_s   = tf.trainable_variables() 
lossL2 = tf.add_n([ tf.nn.l2_loss(v) for v in var_s
                    if 'bias' not in v.name ]) * beta
cost = tf.reduce_mean(cross_entropy )
#error = tf.square(y_pred - y_true)/2 
#cost = tf.reduce_sum(error + error* y_true*5)

optimizer = tf.train.AdamOptimizer().minimize(cost)  #  1e-4
##########################

# ### Performance measures
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))




session = tf.Session()
session.run(tf.global_variables_initializer())


# ### Helper-functions to show performance
feed_dict_test = {x: X_test, y_true: y_test, keep_prob: 1.0}  #.values.reshape([-1,1])
feed_dict_real_test = {x: X_real_test, y_true:y_real_test, keep_prob: 1.0}

def print_accuracy( r_valu = False, feed = feed_dict_test):
    # Use TensorFlow to compute the accuracy.
    acc = session.run(accuracy, feed_dict=feed)
    if r_valu:
        return acc
    # Print the accuracy.
    print("Accuracy on test-set: {0:.1%}".format(acc))

print_accuracy()


saver = tf.train.Saver()  #search tf.saver.tensorflow

# ### Helper-function to perform optimization iterations
batch_size = 200
best_validation_accuracy = 0

# ## optimization

Accuracy on test-set: 49.6%


In [66]:
def optimize( hm_epochs = 10, learning_r = 1e-4, until_acc = False):
    global best_validation_accuracy
    global global_step
    epoch = 0
    while epoch < hm_epochs or best_validation_accuracy<=until_acc:
    		epoch_loss = 0
    		i = 0
    		len_train = X_train.shape[0]
    		while i < len_train:
    			start = i
    			end = i + batch_size
                #global_step++
                
    			batch_x = np.array(X_train[start:end])
    			batch_y = np.array(y_train[start:end])    #.values.reshape([-1,1]))
    			# Put the batch into a dict with the proper names
    			# for placeholder variables in the TensorFlow graph.
    			# Note that the placeholder for y_true_cls is not set
    			# because it is not used during training.

    			feed_dict_train = {x: batch_x,
    							   y_true: batch_y,
    							   keep_prob: 0.8}

    			# Run the optimizer using this batch of training data.
         		# TensorFlow assigns the variables in feed_dict_train
    			# to the placeholder variables and then runs the optimizer.
    			_, c = session.run([optimizer,cost], feed_dict=feed_dict_train)
    			#session.run(optimizer, feed_dict=feed_dict_train)
    			epoch_loss += c

    			i += batch_size

    			if i % 100000 == 0  or i == (len_train - 1):


    				acc_ = print_accuracy(True,feed_dict_real_test)
    				if acc_ > best_validation_accuracy:
                    # Update the best-known validation accuracy.
    					#acc_real = print_accuracy(True,feed_dict_real_test)
    					saver.save(session, 'models/my_model')
    					improved_str = '*'
    					best_validation_accuracy = acc_
    				else:
    					improved_str = ''

    				print('loss: {}, data used: {} / {},  acc: {}{}'.format(epoch_loss/20,i,len_train,acc_,improved_str))
    				epoch_loss = 0
    		epoch +=1


    print('Epoch {} completed out of {}'.format(epoch+1,hm_epochs))

    print('Optimization ended '+ 10*'_' + 'best accuracy: {}'.format(best_validation_accuracy))


In [64]:
#best_validation_accuracy = 0.7
var_s

[<tf.Variable 'weight:0' shape=(58, 400) dtype=float32_ref>,
 <tf.Variable 'bias:0' shape=(400,) dtype=float32_ref>,
 <tf.Variable 'weight_1:0' shape=(400, 1500) dtype=float32_ref>,
 <tf.Variable 'bias_1:0' shape=(1500,) dtype=float32_ref>,
 <tf.Variable 'weight_2:0' shape=(1500, 500) dtype=float32_ref>,
 <tf.Variable 'bias_2:0' shape=(500,) dtype=float32_ref>,
 <tf.Variable 'weight_3:0' shape=(500, 2) dtype=float32_ref>,
 <tf.Variable 'bias_3:0' shape=(2,) dtype=float32_ref>]

In [67]:
optimize(until_acc=0.85)

loss: 13.680325995385648, data used: 100000 / 127560,  acc: 0.7956667542457581
loss: 13.730568033456802, data used: 100000 / 127560,  acc: 0.7940000891685486
loss: 13.739822039008141, data used: 100000 / 127560,  acc: 0.799000084400177
loss: 13.748048855364322, data used: 100000 / 127560,  acc: 0.786333441734314
loss: 13.749595218896866, data used: 100000 / 127560,  acc: 0.7970001101493835
loss: 13.772712957859039, data used: 100000 / 127560,  acc: 0.800000011920929
loss: 13.730621539056301, data used: 100000 / 127560,  acc: 0.7903333902359009
loss: 13.739194665849208, data used: 100000 / 127560,  acc: 0.7986667156219482
loss: 13.744633236527443, data used: 100000 / 127560,  acc: 0.7920000553131104
loss: 13.775992654263973, data used: 100000 / 127560,  acc: 0.7980000972747803
loss: 13.758577513694764, data used: 100000 / 127560,  acc: 0.7860000729560852
loss: 13.768339984118938, data used: 100000 / 127560,  acc: 0.7896667122840881
loss: 13.7796605437994, data used: 100000 / 127560,  ac

KeyboardInterrupt: 

In [72]:
test = pd.read_csv("data/test.csv")

make_sub(test,y_pred_soft_max,session,best_validation_accuracy = best_validation_accuracy)

creating submission results...
INFO:tensorflow:Restoring parameters from models/my_model
Output Created!
Done!


In [69]:
#saver.restore(session,'models/my_model')
feed_dict_real_test = {x: X_real_test, y_true:y_real_test,keep_prob:1}
print_accuracy(feed=feed_dict_test)
a = session.run(y_pred_soft_max,feed_dict_test)
b = a[:10]
c= b.argmax(1)
np.amax(b,1)
[n if c[m] == 0 else 1-n for m,n in zip(range(len(np.amax(b,1))),np.amax(b,1))]
#np.round(b,3)

Accuracy on test-set: 71.0%


[0.43645697832107544,
 0.35772436857223511,
 0.15508240461349487,
 0.70114797,
 0.64638805,
 0.077457785606384277,
 0.46851778030395508,
 0.26048839092254639,
 0.65228772,
 0.48057132959365845]

In [68]:
session.run(cross_entropy,feed_dict_test)

array([ 0.57351166,  0.44273773,  0.16851619, ...,  0.52625763,
        0.26620436,  0.80171454], dtype=float32)

In [70]:
feed_dict_test[y_true][:10]

array([[0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1]])

In [71]:
b[:10]

array([[ 0.43645707,  0.56354302],
       [ 0.35772431,  0.64227563],
       [ 0.15508242,  0.8449176 ],
       [ 0.70114797,  0.298852  ],
       [ 0.64638805,  0.35361198],
       [ 0.0774578 ,  0.92254221],
       [ 0.46851778,  0.53148222],
       [ 0.26048836,  0.73951161],
       [ 0.65228772,  0.34771228],
       [ 0.48057133,  0.51942867]], dtype=float32)

In [17]:
y_train[:30]

array([[0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1]])

In [121]:
ginic(y_50,y_50)

5206.0699999999997